In [12]:
from roboflow import Roboflow
import ultralytics
from ultralytics import YOLO
import os
from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
ultralytics.checks()
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Ultralytics YOLOv8.0.196 🚀 Python-3.8.10 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Setup complete ✅ (44 CPUs, 125.8 GB RAM, 167.4/599.0 GB disk)


In [13]:
# rf = Roboflow(api_key="")
# project = rf.workspace("mlyolotask").project("moouse")
# version = project.version(4)
# dataset = version.download("yolov8")


In [24]:
model=YOLO('yolov8m.pt')
model.train(data='/workspace/Jupyter/binwon/dataset/data.yaml', epochs=60, imgsz=640)

New https://pypi.org/project/ultralytics/8.2.21 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.8.10 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/workspace/Jupyter/binwon/dataset/data.yaml, epochs=60, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visu

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f62bc3f57f0>
fitness: 0.8615650223392791
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.81779,     0.79864,       0.924])
names: {0: 'Flashdisk', 1: 'Mouse', 2: 'cup'}
plot: True
results_dict: {'metrics/precision(B)': 0.9979163090651489, 'metrics/recall(B)': 0.984010118591864, 'metrics/mAP50(B)': 0.9943333333333334, 'metrics/mAP50-95(B)': 0.8468129877843842, 'fitness': 0.8615650223392791}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.14573492501911364, 'inference': 4.33422232929029, 'loss': 0.00036076495521946957, 'postprocess': 0.9247080275886937}

In [25]:

track_history = defaultdict(lambda: [])
model=YOLO('/workspace/Jupyter/binwon/runs/detect/train/weights/best.pt')
names = model.names

video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking.mp4",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2,font_size=2.5)

            for box, cls, track_id,conf in zip(boxes, clss, track_ids,confs):
                annotator.box_label(box, color=colors(int(cls), True), label=f'{names[int(cls)]}:{conf:.2f}')
                
                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

In [20]:
model=YOLO('/workspace/Jupyter/binwon/runs/detect/train3/weights/best.pt')
# results=model(['440px-3-Tasten-Maus_Microsoft.jpg'])
# Run inference on 'bus.jpg' with arguments
model.predict("test.jpg", save=True, imgsz=320, conf=0.5)


image 1/1 /workspace/Jupyter/binwon/test.jpg: 320x256 (no detections), 15.1ms
Speed: 1.2ms preprocess, 15.1ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 256)
Results saved to runs/detect/predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Flashdisk', 1: 'Mouse', 2: 'cup'}
 orig_img: array([[[ 50,  42,  43],
         [ 55,  47,  48],
         [ 65,  57,  58],
         ...,
         [173, 170, 155],
         [171, 168, 153],
         [170, 167, 152]],
 
        [[ 47,  39,  40],
         [ 53,  45,  46],
         [ 63,  55,  56],
         ...,
         [173, 170, 155],
         [173, 170, 155],
         [174, 171, 156]],
 
        [[ 51,  45,  46],
         [ 54,  48,  49],
         [ 60,  54,  55],
         ...,
         [170, 167, 152],
         [173, 170, 155],
         [176, 173, 158]],
 
        ...,
 
        [[166, 181, 197],
         [163, 178, 194],
         [163, 178, 194],
         ...,
         [195, 211, 223],
         [193, 209, 221],
         [191, 207, 219]],
 
        [[164, 179, 195],
         [161, 176, 192],
         [160, 175, 191],
         ...,
    

In [26]:
!python --version

Python 3.10.13
